# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#| export
class DatasetAnalysis():
    """Class to store common functions
    for the analysis of proteomics data"""
    pass

In [ ]:
#| export
class SpectronautProcessor(DatasetAnalysis):
    def __init__(self, file_name='', column_mapping=''):
        self.filename = file_name
        self.column_mapping = self.parse_column_mapping(column_mapping)
        
    #prepare the dataset for analysis    
    def filter_protein_quantification(self, df):
        quant_cols = [n for n in df.columns if 'PG.Quantity' in n]

        # Create a mask DataFrame based on IsSingleHit columns
        mask = df[[n.replace('PG.Quantity', 'PG.IsSingleHit') for n in quant_cols]]
        mask.columns = quant_cols

        # Replace strings with their corresponding boolean values
        replacements = {'Filtered': True, 'False': False, 
                        'True': True, 'FALSE': False, 'TRUE': True}
        mask = mask.replace(replacements)

        # Get the data DataFrame with only the quantification columns
        selection = df[quant_cols]

        # Apply the mask to the data DataFrame
        filtered_selection = selection.mask(mask)

        return filtered_selection
    
    #this function create from a table of:
    #old_column, condition, replica
    #a mepping dataframe. the condition and replica
    #are used to name the new column
    def parse_column_mapping(self, mapping_file):
        # Read the mapping file into a DataFrame
        mapping_df = pd.read_csv(mapping_file, sep='\t')

        # Assert that the DataFrame has exactly 3 columns
        assert mapping_df.shape[1] == 3

        # Replace '.IsSingleHit' with '.Quantity' in the 'col_name' column
        mapping_df['col_name'] = mapping_df['col_name'].str.replace('.IsSingleHit', '.Quantity')

        # Create a new column 'new_col' with the format 'condition.replica'
        mapping_df['new_col'] = mapping_df['condition'] + '.' + mapping_df['replica'].astype(str)

        # Create a dictionary mapping the original column names to the new column names
        mapping_dict = dict(zip(mapping_df['col_name'], mapping_df['new_col']))

        return mapping_dict
    
    #this function apply the logic of getting the dataframe
    #for quantification analysis
    def process(self):
        df = pd.read_csv(self.filename, sep="\t")
        filtered_quantification = self.filter_protein_quantification(df)
        filtered_quantification = filtered_quantification.rename(self.column_mapping,axis=1)
        return filtered_quantification


In [ ]:
# Initialize the class with the Spectronaut output file
# and a table to rename the columns
processor = SpectronautProcessor(
    "../toy_datasets/spectronaut_output.tsv",
    "../toy_datasets/spectronaut_column_mapping.tsv")

# Process the file and get the filtered protein quantification DataFrame
filtered_quantification = processor.process()
filtered_quantification.head()

,CELL34.1,CELL34.2,CELL34.3,CELL37.1,CELL37.2,CELL37.3,CELL40.1,CELL40.2,CELL40.3,SN34.1,SN34.2,SN34.3,SN37.1,SN37.2,SN37.3,SN40.1,SN40.2,SN40.3
0,100761.148438,102093.578125,111970.757812,9.912009e+04,9.892971e+04,104091.296875,9.230105e+04,8.521016e+04,9.303921e+04,105977.328125,175823.1875,172380.53125,117730.5546875,119413.4453125,99135.140625,129270.6875,86490.8203125,NaN
1,527764.312500,500388.437500,550439.875000,5.322265e+05,5.465213e+05,526252.875000,5.184422e+05,5.493845e+05,5.406865e+05,1803379.5,2018498.25,2165891,1756412.75,1795155.625,1982255.25,1549031.375,1549369.25,1828279
2,944657.187500,960022.125000,869661.000000,1.005103e+06,1.223561e+06,260486.875000,4.613028e+05,3.256813e+05,2.587150e+05,NaN,NaN,230601.203125,1692870.25,1343.4593505859375,NaN,NaN,NaN,NaN
3,447414.593750,458718.562500,461863.687500,5.140265e+05,4.905025e+05,490315.968750,5.433065e+05,5.192081e+05,5.276954e+05,38077.0546875,70483.5,116990.828125,70142.5546875,82265.25,81564.125,112112.515625,130479.8203125,93836.3203125
4,860323.937500,824793.812500,827789.562500,1.057513e+06,9.740054e+05,848213.187500,1.076790e+06,1.079964e+06,1.100252e+06,3568417.75,266359.4375,496068.875,1402994.25,904210.75,589144.375,1438730.875,239517.953125,1274326


In [ ]:
filtered_quantification.shape

(49, 18)

In [ ]:
#| export
class DIAnnProcessor(DatasetAnalysis):
    def __init__(self, file_name='', peptides_name='', column_mapping=''):
        self.filename = file_name
        self.column_mapping = self.parse_column_mapping(column_mapping)
        self.peptide = peptides_name
        

    
    #this function apply the logic of getting the dataframe
    #for quantification analysis
    def process(self):
        df = pd.read_csv(self.filename, sep="\t")
        filtered_quantification = self.filter_protein_quantification(df)
        filtered_quantification = filtered_quantification.rename(self.column_mapping,axis=1)
        return filtered_quantification


In [ ]:
#filtered_quantification.tail(50)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()